In [12]:
import numpy as np
import matplotlib.pyplot as plt
from math import log 
from pprint import pprint

Note: a branch with entropy=0 is a leaf node <br/>
a branch with entropy>0 require splitting.

### Equations

$Entropy(s)=\sum_i p(c_i) log_2( p(c_i) )$ <br/>
$information \space gain =Entropy(parent) - \sum_i p(child) * Entropy(child) $ <br/>

$Gini=1-\sum_i p(c_i)^2$

There are two classes (yes, no) <br/>

$P(yes)=number\_of\_yes/total\_data$ <br/>
$P(no)=number\_of\_no/total\_data$ <br/>
$Entropy(s)=\sum_i p(class_i) log_2( p(class_i) ) =P(yes)*log_2 (P(yes) ) + P(no)*log_2(P(no))$ <br/>

$information \space gain =Entropy(parent) - \sum_{group}  p(group) * Entropy(group) $ <br/>

In [13]:
#weather, temperature, humidity, wind, play
data= [
    ['Rain', 'Hot', 'High', 'Weak', 'No'], 
    ['Rain', 'Hot', 'High', 'Strong', 'No'], 
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'Yes'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'No'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Strong', 'No'],
]

#print the data using pretty print (pprint)
data=np.array(data)
print(data)

[['Rain' 'Hot' 'High' 'Weak' 'No']
 ['Rain' 'Hot' 'High' 'Strong' 'No']
 ['Overcast' 'Hot' 'High' 'Weak' 'Yes']
 ['Sunny' 'Mild' 'High' 'Weak' 'Yes']
 ['Sunny' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Sunny' 'Cool' 'Normal' 'Strong' 'No']
 ['Overcast' 'Cool' 'Normal' 'Strong' 'Yes']
 ['Rain' 'Mild' 'High' 'Weak' 'No']
 ['Rain' 'Cool' 'Normal' 'Weak' 'Yes']
 ['Sunny' 'Mild' 'Normal' 'Weak' 'Yes']
 ['Rain' 'Mild' 'Normal' 'Strong' 'Yes']
 ['Overcast' 'Mild' 'High' 'Strong' 'Yes']
 ['Overcast' 'Hot' 'Normal' 'Weak' 'Yes']
 ['Sunny' 'Mild' 'High' 'Strong' 'No']]


In [14]:
def count_class_freq(data):
    """
    data: rows (last column is the lebel) 
    return: dictionary
    """
    classes={} #dictionary
    #your code start
    
    for row in data:
        c=row[-1]
        if c not in classes:
            classes[c]=1
        else:
            classes[c]+=1
            
    #your code end
    return classes

In [15]:
def entropy(data):
    """
    data: rows (last column is the lebel)
    """
    classes=count_class_freq(data) #count classes 
    
    impurity = 0
    #your code start
    classes=count_class_freq(data)
    
    for c in classes:
        prob_of_c = classes[c] / float(len(data))
        impurity -= prob_of_c*  log(prob_of_c, 2)  #2 base log.
    #your code end
    return impurity

In [16]:
def split_cat_data(data, col_id):
    """
    data: rows (last column is the lebel)
    Note: features (column) contains categorical value (like High, Medium, Low)
    
    col_id: column index
    
    return: dictionary
    """
    groups={} #key=category, value=part of the data
    
    #your code start
    for row in data:
        cv=row[col_id]
        if cv in groups:
            groups[cv].append(row)
        else:
            groups[cv]=[row]
    
    #your code end
    for key in groups:
        groups[key]=np.array(groups[key])  #converting to numpy array
    
    
    return groups

In [17]:
def group_probability(groups):
    """
    groups: all the groups
    return: dictionary
    """
    gp={} #dictionary
    #your code start
    nr=0
    for groupname,rows in groups.items():
        nr+=len(rows)
        
    for groupname, group in groups.items():
        gp[groupname]=len(group)/nr
            
    #your code end
    return gp

In [18]:
def information_gain(groups, parent_entropy): 
    """
    groups: a dictionary
    parent_entropy: a float value
    return: information_gain
    """
    
    groups_entropy=0
    gp=group_probability(groups) #claculate probability for each group.
    
    #your code start
    
    for groupnam, rows in groups.items():
        group_entropy=entropy(rows) 
        weighted_entropy= gp[groupnam]*group_entropy
        groups_entropy+=weighted_entropy
    
    #your code start
    
    return parent_entropy - groups_entropy

In [19]:
def get_best_split_cat_column(data):
    """
    data: numpy array (last column is the lebels)
    return: information_gain
    """
    
    num_features=data.shape[1]-1
    et=entropy(data)
    best_id=0
    best_ig=0
    for i in range(num_features):
        groups=split_cat_data(data, i)
        ig=information_gain(groups, et)
        if ig>best_ig:
            best_ig=ig
            best_id=i
            
    return best_id, best_ig

In [20]:
def build_tree_cat(data):
    """
    build decision tree for categorical data
    we will build the tree recursively.
    
    data: numpy array (last column contains the labels)
    expand each decision node until all are leaf
    decision tree consists of decision node and leaf node
    for each decision node, we will add a subtree {}
    for each leaf node, we will stoer a list []
    """
    tree={}
    best_id, best_ig=get_best_split_cat_column(data)
    if best_ig==0: #this is a leaf node.
        return [data[0][-1]]  #return the label.
    
    groups=split_cat_data(data, best_id)
    subtrees={}
#     tree['col']=best_id
    nodename='category?'
    nodename='col_'+str(best_id)+'?'
    for groupname in groups:
        group= groups[groupname]
        subtree=build_tree_cat( group)
        subtrees[groupname]=subtree
    
    tree[nodename]=subtrees
    
    return tree

In [21]:
tree=build_tree_cat(data)
pprint(tree)

{'col_0?': {'Overcast': ['Yes'],
            'Rain': {'col_2?': {'High': ['No'], 'Normal': ['Yes']}},
            'Sunny': {'col_3?': {'Strong': ['No'], 'Weak': ['Yes']}}}}


The decision tree should look like below <br/>

                              col_0?
                            /    |    \
                          /      |      \
                        /        |        \
                      /          |          \
                    /            |            \
                rainy          sunny         overcast
                /                |                \
              /                  |                 \
           col_2?               col_3?              (yes)
           /   \                 / \
         /      \              /     \
      Normal   High         Weak    Strong
       /         \           /         \
      /           \         /            \
    (yes)         (no)    (yes)          (no)
       

<b>Test on new data</b>

In [22]:
def make_decision(tree, x):
    """
    tree: decision tree
    x: a new data (without label)
    
    return: decision
    """
    decision=None
    
    key=list(tree.keys())[0]
    col_id=int( key.split('_')[1].split('?')[0] ) 
    cat=x[col_id]
    branch=tree[key][cat]
    if isinstance(branch, dict): #this is another tree
        return make_decision(branch, x)
    
    decision=branch[0]
    
    return decision

In [23]:
test_x=np.array( ['Rain', 'Mild', 'Normal', 'Weak'] )
decision=make_decision(tree, test_x)
print(decision)

Yes
